## Notebook Objective and Contents:
- In this notebook we have experimented on Fuzzy logic to extract bounding box and start/end endex position value
- Removal of whole image path and Read image using Pillow and save it, named a new column 'raw image'


In [1]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00


In [2]:
!pip install thefuzz

In [3]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import os
import json
from google.colab import data_table
import datasets
from datasets import Dataset
import json
import re
import random
from PIL import Image
from thefuzz import process, fuzz

In [5]:
train_path_main = '/content/drive/MyDrive/docVqa_dataset/train/train_v1.0.json'

# Open the JSON file
with open(train_path_main, 'r') as file:
  # Load the contents of the file into a dictionary
  data_train = json.load(file)

## Convert 'data' list into dataframe by passing into the dataframe
df_train = pd.DataFrame(data_train['data'])

## Making subset of the training dataframe
df_sub_train = df_train.iloc[:50]

base_train_path = '/content/drive/MyDrive/docVqa_dataset/train'

# Define a function to join the base path with the image path
def join_paths(image_path):
    return os.path.join(base_train_path, image_path)

# Apply the function to create a new column with the full image path
df_sub_train['full_path_image'] = df_sub_train['image'].apply(join_paths)


def extract_ocr_info(df, base_train_dir):

  dir_list = os.listdir(base_train_path)
  ocr_base_dir = os.path.join(base_train_path, dir_list[1])

  # Create empty lists for bounding boxes and word lists
  bbox_list = []
  word_list = []

  # Iterate over each row in the DataFrame
  for _, row in df.iterrows():

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Append the bbox and word lists to the global lists
    bbox_list.append(bbox_row)
    word_list.append(word_row)
  # Create new columns in the DataFrame to store the bbox and words lists
  df['bounding_boxes'] = bbox_list
  df['word_list'] = word_list

  df.drop(['questionId', 'image','ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  # Return the updated DataFrame
  return df


<ipython-input-5-248c44dfe9d7>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_train['full_path_image'] = df_sub_train['image'].apply(join_paths)


In [6]:
df_sub_train_ocr = extract_ocr_info(df_sub_train, base_train_path)

<ipython-input-5-248c44dfe9d7>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bounding_boxes'] = bbox_list
<ipython-input-5-248c44dfe9d7>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_list'] = word_list
<ipython-input-5-248c44dfe9d7>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['questionId'

In [7]:
def find_consecutive_indices(df):

  ### case 1
  def case1(words_list,bb_list,output_string):

    output_list = output_string.split()
    output_list_len = len(output_list)

    output_list_start_word = output_list[0]
    output_list_end_word = output_list[output_list_len-1]

    while True:
      output_list_start_word_in_word_list = process.extract(output_list_start_word, words_list, scorer=fuzz.ratio,limit=1)
      output_list_end_word_in_word_list = process.extract(output_list_end_word, words_list, scorer=fuzz.ratio,limit=1)

      start_index = words_list.index(output_list_start_word_in_word_list[0][0])
      end_index = words_list.index(output_list_end_word_in_word_list[0][0])

      if start_index > end_index:
        words_list.pop(end_index)
        continue
      else:
        break

    word_list_values = words_list[start_index:end_index+1]
    bb_list_values = bb_list[start_index:end_index+1]


    return [start_index , end_index], bb_list_values, word_list_values, output_list_len

    ### case 2
  def case2(words_list,output_string):
    output_list = output_string.split()

    unique_matched_words_list = []
    for word in output_list:
      output_list_matched_words_in_word_list = process.extract(word, words_list, scorer=fuzz.ratio)

      matched_words_list = []
      for matched_words in output_list_matched_words_in_word_list:
        matched_words_list.append(matched_words[0])

      matched_words_set = set(matched_words_list)
      unique_matched_words_list.append(matched_words_set)


    index_list=[]
    for words_set in unique_matched_words_list:
      for word in words_set:
        list_size = len(words_list)
        for index in range(list_size):
          # check the condition
          if(words_list[index] == word):

            index_list.append(index)

    index_list_sorted = sorted(index_list)

    numbers = index_list_sorted
    output_list = output_string.split()
    n = len(output_list)
    consecutive_sets = []
    for i in range(len(numbers) - n + 1):
      consecutive_set = numbers[i:i+n]
      if all(consecutive_set[j] + 1 == consecutive_set[j+1] for j in range(n - 1)):

        consecutive_sets.append(consecutive_set)
    return consecutive_sets



  index_for_ver2 = []
  output_strings_v2 = []

  for i in range(0,df.shape[0]):
    words_list_str = str(df["word_list"][i])
    bounding_boxes_str = str(df["bounding_boxes"][i])
    answers_str = str(df["answers"][i])

    words_list = eval(words_list_str)
    bb_list = eval(bounding_boxes_str)
    output = eval(answers_str)
    # words_list = eval(df["word_list"][i])
    # bb_list = eval(df["bounding_boxes"][i])
    # output = eval(df["answers"][i])


    for output_string in output:
      result = case1(words_list,bb_list, output_string)

      if result[3] == len(result[2]):
        result_dict1 = { "Sample_Index" : i,              #Change it to i after final code
                      "actual_answer": output_string,
                      "matched_answer": result[2],
                      "Start_and_End_index": result[0],
                      "bbox_list_values": result[1],
        }

        cell_value = df.at[i, "index_and_bbox_info"]
        cell_value.append(result_dict1)

      else:
        index_for_ver2.append(i)
        output_strings_v2.append(output_string)


  for i,index_ in enumerate(index_for_ver2):

    words_list_str = str(df.at[index_, "word_list"])
    bb_list_str = str(df.at[index_, "bounding_boxes"])

    words_list = eval(words_list_str)
    bb_list = eval(bb_list_str)

    # words_list = eval(df["word_list"][index_])
    # bb_list = eval(df["bounding_boxes"][index_])
    output = [output_strings_v2[i]]


    for output_string in output:
      result = case2(words_list, output_string)

      if result:

        for index_set in result:

            result_dict2 = { "Sample_Index" : index_,                      #todo:add i+2
                                "actual_answer": output_string,
                                "matched_answer":words_list[index_set[0]:index_set[-1]+1],
                                "Start_and_End_index": [index_set[0],index_set[-1]],
                                "bbox_list_values": bb_list[index_set[0]:index_set[-1]+1]}
            cell_value = df.at[index_, "index_and_bbox_info"]
            cell_value.append(result_dict2)

      else:
        pass

  return df


In [8]:
df_sub_train_ocr["index_and_bbox_info"] = [[] for _ in range(len(df_sub_train_ocr))]

<ipython-input-8-a9d3f3734f1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_train_ocr["index_and_bbox_info"] = [[] for _ in range(len(df_sub_train_ocr))]


In [9]:
df_sub_train_ocr_info = find_consecutive_indices(df_sub_train_ocr)

In [10]:
df_sub_train_ocr_info.head()

,question,docId,answers,data_split,full_path_image,bounding_boxes,word_list,index_and_bbox_info
0,what is the date mentioned in this letter?,279,[1/8/93],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[586, 30, 993, 36, 994, 119, 584, 131], [1080...","[Confidential, .., .., RJRT, PR, APPROVAL, DAT...","[{'Sample_Index': 0, 'actual_answer': '1/8/93'..."
1,what is the contact person name mentioned in l...,279,"[P. Carter, p. carter]",train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[586, 30, 993, 36, 994, 119, 584, 131], [1080...","[Confidential, .., .., RJRT, PR, APPROVAL, DAT...","[{'Sample_Index': 1, 'actual_answer': 'P. Cart..."
2,Which corporation's letterhead is this?,280,[Brown & Williamson Tobacco Corporation],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI...","[{'Sample_Index': 2, 'actual_answer': 'Brown &..."
3,Who is in cc in this letter?,280,[T.F. Riehl],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI...","[{'Sample_Index': 3, 'actual_answer': 'T.F. Ri..."
4,what is the subject of this letter?,280,[Review of existing Brainstorming Ideas/483],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI...","[{'Sample_Index': 4, 'actual_answer': 'Review ..."


In [11]:
df_sub_train_ocr_info["index_and_bbox_info"][1]

[{'Sample_Index': 1,
  'actual_answer': 'P. Carter',
  'matched_answer': ['P.', 'CARTER'],
  'Start_and_End_index': [20, 21],
  'bbox_list_values': [[445, 521, 490, 519, 492, 577, 447, 578],
   [501, 519, 658, 513, 660, 568, 503, 577]]},
 {'Sample_Index': 1,
  'actual_answer': 'p. carter',
  'matched_answer': ['P.', 'CARTER'],
  'Start_and_End_index': [20, 21],
  'bbox_list_values': [[445, 521, 490, 519, 492, 577, 447, 578],
   [501, 519, 658, 513, 660, 568, 503, 577]]}]

In [12]:
## Convert dataframe into dataset object
hf_sub_train_ocr_info = Dataset.from_pandas(df_sub_train_ocr_info)

In [13]:
hf_sub_train_ocr_info

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'index_and_bbox_info'],
    num_rows: 50
})

In [14]:
# Define a function to open and save images
def process_image(hfsample):
    image_path = hfsample['full_path_image']
    image = Image.open(image_path)
    hfsample['image_raw'] = image
    del hfsample['full_path_image']
    return hfsample

# Apply the function to each row in the dataset
hf_sub_train_ocr_info_imgraw = hf_sub_train_ocr_info.map(process_image)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [15]:
hf_sub_train_ocr_info_imgraw

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'bounding_boxes', 'word_list', 'index_and_bbox_info', 'image_raw'],
    num_rows: 50
})

In [16]:
hf_sub_train_ocr_info_imgraw["index_and_bbox_info"][0]

[{'Sample_Index': 0,
  'Start_and_End_index': [8, 8],
  'actual_answer': '1/8/93',
  'bbox_list_values': [[400, 258, 556, 261, 555, 332, 399, 329]],
  'matched_answer': ['1/8/13']}]

In [17]:
hf_sub_train_ocr_info_imgraw["index_and_bbox_info"][5]

[{'Sample_Index': 5,
  'Start_and_End_index': [320, 320],
  'actual_answer': '499150498',
  'bbox_list_values': [[1112, 1806, 1338, 1802, 1338, 1837, 1111, 1840]],
  'matched_answer': ['499150498']}]

In [18]:
hf_sub_train_ocr_info_imgraw

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'bounding_boxes', 'word_list', 'index_and_bbox_info', 'image_raw'],
    num_rows: 50
})